In [1]:
from RewardingVisualDoubt import infrastructure

infrastructure.make_ipython_reactive_to_changing_codebase()


from RewardingVisualDoubt import (
    dataset,
    green,
    evaluation,
    training,
    vllm,
    prompter,
    inference,
    response,
    reward,
    shared,
)

import accelerate
import dataclasses
import torch
import functools
import pathlib as path
import os
import itertools
import time

from tqdm import tqdm

import typing as t

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Fetching 69 files:   0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
BATCH_SIZE = 24
SELECTED_ADAPTER_PATH = None

device, device_str = shared.get_device_and_device_str()

model = vllm.load_pretrained_llava_model_for_ppo_training_with_lora_adapters(
    device_str=device_str,
    llava_model_path=vllm.RadialogMergedLlavaModelPath.REPORT_GENERATION_WITH_CONFIDENCE_SFT.value,
    precision="4bit",
    adapter_path=SELECTED_ADAPTER_PATH,
)
tokenizer = vllm.load_pretrained_llava_tokenizer_with_image_support(
    model_base=vllm.LLAVA_BASE_MODEL_NAME
)

model.config.padding_side = "left"


selected_prompter_fn = prompter.build_report_generation_prompt_with_response_and_confidence_for_sft
prompter_ = functools.partial(
    selected_prompter_fn,
    is_for_inference=True,
)

testset_in_dist = dataset.get_in_distribution_report_generation_test_set(tokenizer, prompter_)
dataloader_in_dist = dataset.get_mimic_cxr_llava_model_input_dataloader(
    testset_in_dist,
    batch_size=BATCH_SIZE,
    padding_tokenizer=vllm.load_pretrained_llava_tokenizer_with_image_support(
        for_use_in_padding=True
    ),
    num_workers=8,
)